In [1]:
from rand_archive import Reader, Writer, Header
from src.dataloader.types import VideoSample, Video
import pypeln as pl

In [2]:
from numpy import ndarray
from typing import List
from av.video.frame import VideoFrame
import av

def ser_model(frames: List[ndarray]) -> bytes:
    buf = BytesIO()
    container = av.open(buf, mode='w', format='h264')
    stream = container.add_stream(
        'h264',
        options={
            'preset': 'ultrafast',
            'tune': 'fastdecode',
            'crf': '28',
        }
    )
    stream.pix_fmt = 'yuv420p'
    stream.width = frames[0].shape[1]
    stream.height = frames[0].shape[0]
    for frame in frames:
        frame = VideoFrame.from_ndarray(frame, format='rgb24')
        for packet in stream.encode(frame):
            container.mux(packet)
    for packet in stream.encode():
        container.mux(packet)
    container.close()
    return buf.getvalue()

In [3]:
import pickle
from io import BytesIO

def convert_to_video(x):
    key, b = x
    d = pickle.loads(b)
    md = d['md']
    cls = md.pop('cls')
    md.pop('key')
    return VideoSample(
        key=key,
        cls=cls,
        video=Video.from_path(BytesIO(d['video'])),
        extra=md
    )

In [4]:
vid = convert_to_video(next(iter(Reader().open_file('test/data/test.raa'))))

In [5]:
%%time
_ = ser_model(vid.video.frames)

CPU times: user 41.1 ms, sys: 32.3 ms, total: 73.4 ms
Wall time: 173 ms


In [6]:
%%time
_ = vid.model_dump()

CPU times: user 42.7 ms, sys: 14.4 ms, total: 57.1 ms
Wall time: 51.8 ms


In [3]:
data = (
    iter(Reader().open_file('data/test.raa'))
    | pl.thread.map(convert_to_video, workers=4)
)

In [ ]:
from tqdm import tqdm
w = Writer('data/val.raa', max_header_size=1585728)
for x in tqdm(data):
    w.write(x.key, pickle.dumps(x.model_dump()))
w.close()

In [ ]:
from tqdm import tqdm
for x in tqdm(data):
    x.model_dump()

150it [00:19,  4.45it/s]